# MIE1628 Project 5 - B

### PhishGuard: Machine Learning-Based Detection of Phishing Websites

Description of the our project  
- Phishing websites pose a major cybersecurity threat by imitating legitimate websites to steal sensitive information such as passwords, credit card numbers, and banking credentials.  
- As online transactions and digital communications become increasingly integral to daily life, the risk associated with phishing attacks has grown substantially.  
- This project focuses on addressing this issue by developing a machine learning model that can accurately classify websites as legitimate or phishing based on a range of behavioral and structural website attributes.  
- Our preject is aim to improve browser security tools, empower users with real-time phishing detection alerts, and reduce the number of successful phishing attempts in the digital ecosystem.

### Data preprocessing

- Load the dataset and inspect its structure
- Split the data into features and target labels
- Split the dataset into training and testing sets

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

In [3]:
# Load ARFF file
with open('phishing.arff', 'r') as f:
    data = arff.load(f)

# Convert to DataFrame
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])

# Quick look at the data
print(df.head())
print(df.info())


FileNotFoundError: [Errno 2] No such file or directory: 'phishing.arff'